### Script Python per Unire e Filtrare Dati NIC da Excel

Questo script unisce e filtra dati mensili dell’Indice Generale NIC provenienti da due file Excel (anni 1996-2015 il primo, 2016-2024 il secondo), entrambi a frequenza mensile, destagionalizzati e con base 2015=100.

Estrae la serie desiderata da ciascun file, la trasforma in formato lungo, unisce i dati, rimuove duplicati, ordina cronologicamente e salva il risultato finale in un file CSV pronto per l’analisi.

In [ ]:
import pandas as pd
import os
from datetime import datetime

# --- PARAMETRI DI CONFIGURARZIONE ---
# Percorsi completi dei file Excel di input
file_excel_1_path = "/Users/tommaso/Desktop/tesi-inflation-gt/ISTAT_data/Reconstruction NIC base 2015 (1996-2015).xls"
file_excel_2_path = "/Users/tommaso/Desktop/tesi-inflation-gt/ISTAT_data/Principali dati (IT1,167_744_DF_DCSP_NIC1B2015_1,1.0).xlsx"

# Directory di output specificata
output_directory = "/Users/tommaso/Desktop/tesi-inflation-gt/ISTAT_data"
# Nome del file CSV di output
output_filename = "NIC_unificato_2004_2024_finale_v3.csv"
# --- FINE DEI PARAMETRI DI CONFIGURAZIONE ---

# --- FUNZIONI DI PROCESSAMENTO DATI ---
def process_file_1(file_path):
    """
    Processa il primo file Excel (1996-2015).
    Estrae i dati dell'Indice Generale NIC, li trasforma da wide a long,
    e seleziona il periodo da Gennaio 2004 a Dicembre 2015.
    """
    print(f"--- Processamento File 1: {file_path} ---")
    engine_to_use = None
    if file_path.endswith('.xls'):
        engine_to_use = 'xlrd'
        print("Utilizzo di 'xlrd' engine per il file .xls.")
    
    try:
        df1 = pd.read_excel(file_path, sheet_name="IT", header=10, engine=engine_to_use)
        print(f"File 1 (foglio 'IT') caricato. Nomi colonne letti (primi 10): {df1.columns.tolist()[:10]}")
    except FileNotFoundError:
        print(f"ERRORE: File 1 non trovato: {file_path}")
        return None
    except Exception as e:
        print(f"ERRORE durante la lettura del File 1 (foglio 'IT', header=10): {e}")
        return None
    
    try:
        riga_indice_generale = df1[
            (df1['ECOICOP'].astype(str).str.strip() == "00") &
            (df1['livello'].astype(str).str.strip() == "Gen.") &
            (df1['denominazioni 2016'].astype(str).str.strip() == "Indice generale NIC")
        ].copy()
    except KeyError as ke:
        print(f"ERRORE: Una delle colonne chiave ('ECOICOP', 'livello', 'denominazioni 2016') non è stata trovata. "
              f"Verifica che 'header=10' sia corretto e che i nomi colonna corrispondano. Errore: {ke}")
        return None

    if riga_indice_generale.empty:
        print("ERRORE: Riga 'Indice generale NIC' non trovata nel File 1 con i criteri specificati.")
        return None
    
    print(f"Riga 'Indice generale NIC' trovata nel File 1.")

    # Identifico le colonne dei mesi che sono già state lette come datetime
    colonne_datetime_file1 = [col for col in riga_indice_generale.columns if isinstance(col, datetime)]
    
    if not colonne_datetime_file1:
        print("ERRORE: Nessuna colonna di tipo datetime (per i mesi) trovata nel File 1 dopo il caricamento.")
        print(f"Tipi di dato delle colonne (primi 10): {riga_indice_generale.dtypes[:10]}")
        return None

    print(f"Trovate {len(colonne_datetime_file1)} colonne di tipo datetime nel File 1.")

    # Filtro queste colonne datetime per l'intervallo Gen 2004 - Dic 2015
    data_inizio_periodo_file1 = datetime(2004, 1, 1)
    data_fine_periodo_file1 = datetime(2015, 12, 1) # Consideriamo il primo del mese

    colonne_mesi_da_estrarre_file1 = [
        col for col in colonne_datetime_file1 
        if data_inizio_periodo_file1 <= col <= data_fine_periodo_file1
    ]

    if not colonne_mesi_da_estrarre_file1:
        print(f"ERRORE: Nessuna colonna datetime trovata nell'intervallo {data_inizio_periodo_file1.strftime('%Y-%m')} - {data_fine_periodo_file1.strftime('%Y-%m')} nel File 1.")
        print(f"Esempio di colonne datetime trovate (prime 5): {colonne_datetime_file1[:5]}")
        return None
        
    print(f"Colonne mese (datetime) identificate e filtrate nel File 1 (conteggio): {len(colonne_mesi_da_estrarre_file1)}")
    print(f"Esempio colonne filtrate (prime 3): {colonne_mesi_da_estrarre_file1[:3]}")
    print(f"Esempio colonne filtrate (ultime 3): {colonne_mesi_da_estrarre_file1[-3:]}")


    # Seleziono solo le colonne identificate (che sono già datetime) e le prime colonne identificative
    colonne_identificative = ['ECOICOP', 'livello', 'denominazioni 2016']
    # Mi assicuro che le colonne identificative esistano prima di usarle
    existing_id_cols = [col for col in colonne_identificative if col in riga_indice_generale.columns]
    
    dati_selezionati = riga_indice_generale[existing_id_cols + colonne_mesi_da_estrarre_file1]

    # Trasformo da wide a long (unpivot)
    # id_vars saranno le colonne che non devono essere "unpivoted"
    df1_long = dati_selezionati.melt(
        id_vars=existing_id_cols, 
        value_vars=colonne_mesi_da_estrarre_file1,
        var_name="Periodo", # Questa colonna conterrà direttamente gli oggetti datetime
        value_name="Valore_NIC"
    )
    
    # 'Periodo' è già datetime, non serve ulteriore conversione da stringa.
    df1_long.dropna(subset=['Periodo', 'Valore_NIC'], inplace=True)

    print(f"File 1 processato. Righe dopo melt e dropna: {len(df1_long)}")
    if not df1_long.empty:
        print(f"Test File 1 - Periodo min: {df1_long['Periodo'].min()}, Periodo max: {df1_long['Periodo'].max()}")
    return df1_long[['Periodo', 'Valore_NIC']]

def process_file_2(file_path):
    """
    Processa il secondo file Excel (2016-2024).
    Estrae i dati dell'Indice Generale NIC e li trasforma da wide a long.
    """
    print(f"\n--- Processamento File 2: {file_path} ---")
    try:
        df2 = pd.read_excel(file_path, sheet_name="M IT 39 4", header=6)
        print(f"File 2 (foglio 'M IT 39 4') caricato. Nomi colonne (primi 10): {df2.columns.tolist()[:10]}")
    except FileNotFoundError:
        print(f"ERRORE: File 2 non trovato: {file_path}")
        return None
    except Exception as e:
        print(f"ERRORE durante la lettura del File 2 (foglio 'M IT 39 4', header=6): {e}")
        return None

    desc_col_name = df2.columns[0] 
    print(f"Colonna descrittiva identificata nel File 2: '{desc_col_name}'")
    
    try:
        # Mi assicurato che la colonna descrittiva sia trattata come stringa per il confronto
        riga_indice_generale = df2[df2[desc_col_name].astype(str).str.contains("00 Indice generale", na=False, case=False, regex=False)].copy()
    except KeyError:
        print(f"ERRORE: Colonna descrittiva '{desc_col_name}' non trovata nel File 2 dopo aver impostato header=6.")
        return None

    if riga_indice_generale.empty:
        print(f"ERRORE: Riga '00 Indice generale' non trovata nel File 2 (colonna '{desc_col_name}').")
        print(f"Contenuto della colonna '{desc_col_name}' (prime 5 righe del df2):\n{df2[desc_col_name].head()}")
        return None
    
    print(f"Riga '00 Indice generale' trovata nel File 2.")

    colonne_dati_valide_per_melt = []
    for col in riga_indice_generale.columns[1:]:
        col_name_str = str(col).strip()
        if col_name_str:
            try:
                datetime.strptime(col_name_str, "%Y-%m") # Verifica del formato AAAA-MM
                colonne_dati_valide_per_melt.append(col)
            except ValueError:
                if isinstance(col, datetime): # Se pandas l'ha già parsato come datetime
                     colonne_dati_valide_per_melt.append(col)
                else:
                    print(f"Attenzione: colonna '{col}' nel File 2 scartata (nome '{col_name_str}' non è AAAA-MM o datetime).")
        else:
            print(f"Attenzione: colonna con nome vuoto/spazi scartata nel File 2.")

    if not colonne_dati_valide_per_melt:
        print("ERRORE: Nessuna colonna mese valida (formato AAAA-MM o datetime) trovata nel File 2.")
        return None

    print(f"Colonne mese identificate nel File 2 (conteggio): {len(colonne_dati_valide_per_melt)}")
    dati_selezionati = riga_indice_generale[colonne_dati_valide_per_melt]
    
    df2_long = dati_selezionati.melt(var_name="Periodo_Testo", value_name="Valore_NIC")

    def convert_periodo_file2(periodo_obj):
        if isinstance(periodo_obj, datetime):
            return periodo_obj
        periodo_str = str(periodo_obj).strip()
        try:
            return datetime.strptime(periodo_str, "%Y-%m")
        except ValueError:
            print(f"Attenzione: formato data non riconosciuto per '{periodo_obj}' nel File 2.")
            return None
            
    df2_long['Periodo'] = df2_long['Periodo_Testo'].apply(convert_periodo_file2)
    df2_long.dropna(subset=['Periodo', 'Valore_NIC'], inplace=True) # Rimuovo anche se Valore_NIC è NaN

    print(f"File 2 processato. Righe dopo melt e dropna: {len(df2_long)}")
    if not df2_long.empty:
        print(f"Test File 2 - Periodo min: {df2_long['Periodo'].min()}, Periodo max: {df2_long['Periodo'].max()}")
    return df2_long[['Periodo', 'Valore_NIC']]

# --- ESECUZIONE DELLO SCRIPT ---
df1_processed = process_file_1(file_excel_1_path)
df2_processed = process_file_2(file_excel_2_path)

if df1_processed is not None and not df1_processed.empty and \
   df2_processed is not None and not df2_processed.empty:
    
    print("\n--- Unione e Finalizzazione ---")
    print(f"Righe da File 1: {len(df1_processed)}, Righe da File 2: {len(df2_processed)}")
    df_unificato = pd.concat([df1_processed, df2_processed], ignore_index=True)

    df_unificato['Valore_NIC'] = pd.to_numeric(df_unificato['Valore_NIC'], errors='coerce')
    df_unificato.dropna(subset=['Valore_NIC', 'Periodo'], inplace=True)

    print("Rimozione di eventuali duplicati basati sul Periodo (mantenendo il primo)...")
    df_unificato.drop_duplicates(subset=['Periodo'], keep='first', inplace=True)

    print("Ordinamento cronologico del dataset unificato...")
    df_unificato.sort_values('Periodo', inplace=True)
    
    print(f"Dataset unificato creato. Numero righe totali: {len(df_unificato)}")
    if not df_unificato.empty:
        print(f"Periodo coperto: da {df_unificato['Periodo'].min().strftime('%Y-%m-%d')} a {df_unificato['Periodo'].max().strftime('%Y-%m-%d')}\n")

        data_inizio_filtro = pd.to_datetime("2004-01-01")
        data_fine_filtro = pd.to_datetime("2024-12-31")

        print(f"Filtraggio dati da {data_inizio_filtro.strftime('%Y-%m-%d')} a {data_fine_filtro.strftime('%Y-%m-%d')}...")
        
        df_filtrato = df_unificato[
            (df_unificato['Periodo'] >= data_inizio_filtro) &
            (df_unificato['Periodo'] <= data_fine_filtro)
        ].copy()

        print(f"Dataset filtrato. Numero righe: {len(df_filtrato)}")
        if not df_filtrato.empty:
            print(f"Periodo filtrato coperto: da {df_filtrato['Periodo'].min().strftime('%Y-%m-%d')} a {df_filtrato['Periodo'].max().strftime('%Y-%m-%d')}")
            print(f"Prime 5 righe del dataset filtrato:\n{df_filtrato.head()}\n")
            print(f"Ultime 5 righe del dataset filtrato:\n{df_filtrato.tail()}\n")

            try:
                os.makedirs(output_directory, exist_ok=True)
                print(f"Directory di output '{output_directory}' verificata/creata.")
            except OSError as e:
                print(f"ERRORE: Impossibile creare la directory di output '{output_directory}': {e}")
                print("Salvataggio annullato.")
            else:
                full_output_path = os.path.join(output_directory, output_filename)
                try:
                    df_filtrato_save = df_filtrato.copy()
                    # Converto esplicitamente Periodo in datetime
                    df_filtrato_save['Periodo'] = pd.to_datetime(df_filtrato_save['Periodo'])
                    # Formatto la colonna Periodo come AAAA-MM
                    df_filtrato_save['Periodo'] = df_filtrato_save['Periodo'].dt.strftime('%Y-%m')
        
                    df_filtrato_save.to_csv(full_output_path, index=False)
                    print(f"Dataset finale salvato con successo in: {full_output_path}")
                except Exception as e:
                    print(f"ERRORE durante il salvataggio del file CSV '{full_output_path}': {e}")

        else:
            print("Il dataset filtrato è vuoto. Controlla le date di filtro e i dati originali.")
    else:
        print("Il dataset unificato è vuoto. Problemi durante il processamento dei file.")
else:
    print("\nUno o entrambi i file Excel non sono stati processati correttamente o non hanno prodotto dati validi. Impossibile procedere con l'unione.")
    if df1_processed is None or df1_processed.empty :
        print("Problema specifico con il File 1 o il suo output.")
    if df2_processed is None or df2_processed.empty:
        print("Problema specifico con il File 2 o il suo output.")


print("\nScript terminato.")



--- Processamento File 1: /Users/tommaso/Desktop/tesi-inflation-gt/ISTAT_data/Reconstruction NIC base 2015 (1996-2015).xls ---
Utilizzo di 'xlrd' engine per il file .xls.
File 1 (foglio 'IT') caricato. Nomi colonne letti (primi 10): ['ECOICOP', 'livello', 'denominazioni 2016', datetime.datetime(1996, 1, 1, 0, 0), datetime.datetime(1996, 2, 1, 0, 0), datetime.datetime(1996, 3, 1, 0, 0), datetime.datetime(1996, 4, 1, 0, 0), datetime.datetime(1996, 5, 1, 0, 0), datetime.datetime(1996, 6, 1, 0, 0), datetime.datetime(1996, 7, 1, 0, 0)]
Riga 'Indice generale NIC' trovata nel File 1.
Trovate 240 colonne di tipo datetime nel File 1.
Colonne mese (datetime) identificate e filtrate nel File 1 (conteggio): 144
Esempio colonne filtrate (prime 3): [datetime.datetime(2004, 1, 1, 0, 0), datetime.datetime(2004, 2, 1, 0, 0), datetime.datetime(2004, 3, 1, 0, 0)]
Esempio colonne filtrate (ultime 3): [datetime.datetime(2015, 10, 1, 0, 0), datetime.datetime(2015, 11, 1, 0, 0), datetime.datetime(2015, 12, 1